# Intermediate topics

If you are already familiar with the Python basics, this notebook will introduce you to some more advanced topics.
Neither of these topics is required to complete the exercises, but they may be useful to you in the future.

This notebook is not a complete introduction to these topics, but rather a quick overview of some of the most useful features.
If you want to learn more, you can find many tutorials online.

We will cover:

- Dict-comprehensions
- Lambda functions
- Type hints
- Keyword arguments
- `*args` and `**kwargs`
- Dataclasses
- Decorators
- Utilities from the standard library

## Dict-comprehension

We previously introduced `list`-comprehension:

In [ ]:
mylist = [2, 1, 3, 4, 5]
mysquares = [x**2 for x in mylist]
mysquares

There is also `dict`-comprehension, with various flavors:

In [ ]:
mydict = {"a": 1, "b": 2, "c": 3}

# Iterate over dict items
mysquaresdict = {key: value**2 for (key, value) in mydict.items()}

# Iterate over dict items and modify keys
mydictwithnewkeys = {key.upper(): value**2 for (key, value) in mydict.items()}

# Iterate over a list or other iterable
mydictfromlist = {x: x**2 for x in mylist}

print(mysquaresdict)
print(mydictwithnewkeys)
print(mydictfromlist)

## Lambda functions

Lambda functions are a way to define functions "inline", without giving them a name.
This can be useful when we deal with another function that takes a function as an argument.
The syntax is:

```python
lambda <arguments>: <expression>
```

The expression is evaluated and returned when the function is called.
Example:

In [ ]:
def process_list(values, function):
    return [function(x) for x in values]


mylist = [2, 1, 3, 4, 5]
print(process_list(mylist, lambda x: x + 2))
print(process_list(mylist, lambda x: x**2))

## Type hints

Python is a dynamically typed language, which means that the type of a variable is not known at compile time.
This is in contrast to statically typed languages like C, where the type of a variable must be declared before it is used.
As a relatively recent addition to the language, Python now supports type hints.
These are not enforced by the interpreter, but can be used by external tools to check for type errors.
Maybe more importantly, they can be used to document the expected types of arguments and return values, improving readability.
Over the past years, type hints have become increasingly popular, and are now used in many popular libraries, to the point where they are almost expected.

Example:

In [ ]:
def format_info(name: str, age: int) -> str:
    return f"{name} is {age} years old"

- Type hints for function arguments are written after the argument name, separated by a colon:
  - `name` is a string
  - `age` is an integer
- The return type is written after an arrow `->`:
  - The function returns a string

Note that the type hints are literally *hints* and are not enforced by the interpreter, so the following code will run without errors:

In [ ]:
format_info("John", 25.5)

In [ ]:
format_info(11, [1, 2, 3])


Apart from single types like `int`, `float`, `str`, `list`, `dict`, etc., we can also use combinations of types.
Note that the `|` operator means 'OR', i.e. that the type can be one or the other.

Example:

In [ ]:
import math


def mysqrt(x: list[int | float]) -> list[float]:
    """Take a list of integers or floats and return a list of their square roots"""
    return [math.sqrt(y) for y in x]

## Keyword arguments

When calling a function, we can specify the arguments by position or by name.
The latter is called *keyword arguments*.
Keyword arguments can be used to make the code more readable, and to specify default values for arguments.
Using keyword arguments for all arguments is a good practice (apart from cases such as function with a single arguments), as it makes the code more robust to changes in the function signature and improves readability:

In [ ]:
def format_info(name: str, age: int) -> str:
    return f"{name} is {age} years old"


format_info(name="John", age=25)

## \*args and \*\*kwargs

Sometimes we want to write a function that takes an arbitrary number of arguments.
This can be done using `*args` and `**kwargs`.
The names `args` and `kwargs` are not special, but are commonly used.
Example:

In [ ]:
def average_age(*ages: int) -> float:
    return sum(ages) / len(ages)


def format_family_info(**ages: int) -> str:
    info = ", ".join([f"{name} is {age} years old" for name, age in ages.items()])
    return info + f". Their average age is {average_age(*ages.values())}."


format_family_info(John=25, Jane=24, Jack=17)

We can also use `*` and `**` when calling a function, to unpack a list or dictionary into arguments:

In [ ]:
family = {"Jack": 17, "Jill": 15}
print(f"Average age: {average_age(*family.values())}")
print(format_family_info(**family))

## Dataclasses

[Dataclasses](https://docs.python.org/3/library/dataclasses.html) are a convenient way to create classes that are mostly used to store data.
Even if you are not familiar with classes in Python, you can probably understand the following example.
For more details, see, for example, this video introduction: [If you're not using Python DATA CLASSES yet, you should](https://www.youtube.com/watch?v=vRVVyl9uaZc).

In [ ]:
from dataclasses import dataclass


@dataclass
class Experiment:
    name: str
    date: str
    temperature: float
    pressure: float

`@dataclass` is a so-called decorator that adds convenient functionality to the class.
We can use our class `Experiment` as follows:

In [ ]:
exp1 = Experiment(
    name="my first experiment", date="2020-01-01", temperature=20.0, pressure=1.0
)
exp2 = Experiment(
    name="my second experiment", date="2020-01-02", temperature=21.0, pressure=1.1
)
print(exp1)
print(exp2)
# Access fields
print(exp1.date)

Dataclasses can also have default arguments.
For mutable default argument such as a list, we need to use `field(default_factory=list)`:

In [ ]:
from dataclasses import dataclass, field


@dataclass
class Experiment:
    date: str
    location: str = "ESS"
    comments: list = field(default_factory=list)


# Default location and comments
print(Experiment(date="2020-01-01"))
# Override default location, keep default comments
print(Experiment(date="2020-01-01", location="ILL"))
# Default location, override default comments
print(Experiment(date="2020-01-01", comments=["test1", "test2"]))

## Decorators

Decorators are a way to modify the behavior of a function.
They are written as functions that take a function as an argument, and return a new function.
The syntax is:

In [ ]:
def mydecorator(func):
    def wrapper(*args, **kwargs):
        # Do something before calling the function
        print(
            f"Your function {func.__name__} is about to be called with arguments {args} and {kwargs}"
        )
        # Call the function
        result = func(*args, **kwargs)
        # Do something after calling the function
        print(f"Your function {func.__name__} was called")
        return result

    return wrapper


@mydecorator
def myfunction():
    print("Hello world")


@mydecorator
def myfunction2(x):
    print(f"Hello {x}")


myfunction()
myfunction2("John")

## Utilities from the standard library

When you are writing Python code, you should always check if there is a utility in the standard library that does what you want.
More often than not, there is.
Here are some examples:

- [Built-in functions](https://docs.python.org/3/library/functions.html) such as `map`, `filter`, `enumerate`, `zip`, and `reduce`.
- [itertools](https://docs.python.org/3/library/itertools.html).
- [functools](https://docs.python.org/3/library/functools.html), in particular [functools.partial](https://docs.python.org/3/library/functools.html#functools.partial).